In [162]:
import pandas as pd
from tenacity import retry, wait_random_exponential, stop_after_attempt
import logging
import json
from tqdm.auto import tqdm

In [50]:
import os
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [51]:
# import openai
# openai.api_key = os.getenv("OPENAI_API_KEY")

In [209]:
# new api
from openai import OpenAI
from openai import OpenAIError

client = OpenAI(
  api_key=os.getenv('OPENAI_API_KEY'),  # this is also the default, it can be omitted
)

In [216]:
raw_preds = pd.read_csv("../submissions/multibert_ukru_lingua_00_ot_stoprows4_n.csv", converters={"locations": eval})
df_test = pd.read_csv("../data/nlp-ua-locations-extractions/test.csv")
raw_preds['text'] = df_test['text']

In [144]:
system_message = """
You are a professional datasets labelist. Your job is labeling datasets for further usage in machine learning tasks. Now you are working on the dataset for NER task, which focuses on location extraction. 
You will be given a couple of text samples in Ukrainian and Russian languages along with the set of possible location entities in them. Your task is to inspect each suggested entity and make a verdict on whether it's a valid location entity in the given context or not and explain your decision.  
""" 

user_message_example = """
Text: "РОЗШУКУЮТЬСЯ власники песика зображеного на фото який близько 1-2 тижнів перебуває на території Дарницького району (Бориспільська, 9, корпус 54, територія колишнього радіозаводу)

🐶можливо хтось впізнає свого домашнього улюбленця та забере додому🏠

🟥Характерні ознаки песика:
- колір чорно-коричневий;
- ошейник зеленого кольору (хакі);
-хвостик на кінчику зі стрижкою.

☝🏻Звернутися можна до мешканців багатоквартирного будинку за адресою Бориспільська, 9, які його тут бачать щодня. 
У випадку  якщо впізнали песика та не знаєте де знаходиться територія телефонуйте за номером 0932626376."
Entities: ['Дарницького району', 'Бориспільська, 9', 'корпус 54', 'Бориспільська, 9']
"""

assisstant_message_example = """
[[
    {
        'entity': 'Дарницького району'
        'valid': "YES",
        'explanation': "Refers to the district of city in which a puppy was lost"
    },
    {
        'entity': 'Бориспільська, 9'
        'valid': "NO",
        'explanation': "only part of the full address 'Бориспільська, 9, корпус 54' which is given in the text"
    },
    {
        'entity': 'корпус 54'
        'valid': "NO",
        'explanation': "only part of the full address 'Бориспільська, 9, корпус 54' which is given in the text"
    },
    {
        'entity': 'Бориспільська, 9'
        'valid': "YES",
        'explanation': "address of the mentioned citizen, it's a complete one in the context"
    }
]] 
"""

def user_message(text, entities):
    return f"""
    Text: "{text}"
    Entities: {entities} 
    """

In [145]:
def generate_functions() -> list:
    return [
        {
            "name": "extract_entities",
            "description": "Select valid entities from the full explanations",
            "type": "object",
            "parameters": {
                "type": "object",
                "properties": {
                    "entities": {
                        "description": "entities reports",
                        "type": "array",
                        "items": {
                            "type": "object",
                                "properties": {
                                    "entity": {
                                        "type": "string"
                                    },
                                    "valid": {
                                        "type": "string"
                                    },
                                    "explanation": {
                                        "type": "string"
                                    }
                                }
                        }
                    }
                }
            },
        }
    ]

In [146]:
def extract_entities(entities):
    res = []
    for entity in entities:
        if entity['valid'] == "YES":
            res.append(entity['entity'])
        
    return res

In [229]:
# @retry(wait=wait_random_exponential(min=1, max=4), stop=stop_after_attempt(2))
def run_openai_task(text, entities):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message_example},
        {"role": "assistant", "content": assisstant_message_example},
        {"role": "user", "content": user_message(text=text, entities=entities)}
    ]

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
            functions=generate_functions(),
            function_call={"name": "extract_entities"}, 
            temperature=0,
            frequency_penalty=0,
            presence_penalty=0,
        )

        response_message = response.choices[0].message

        available_functions = {"extract_entities": extract_entities}  
        function_name = response_message.function_call.name

        function_to_call = available_functions[function_name]
        logging.info(f"function_to_call: {function_to_call}")

        function_args = json.loads(response_message.function_call.arguments)['entities']
        logging.info(f"function_args: {function_args}")

        function_response = function_to_call(function_args)
        response = response.model_dump()
    except Exception as error:
        print(error)
        function_response = entities
        response = {"error": error}

    return {"model_response": response,
            "function_response": function_response}

In [237]:
locations = []
responses = []

for i, row in tqdm(raw_preds.iterrows(), total=len(raw_preds)):
    if len(row['locations']) == 0:
        locations.append([])
        responses.append({})
        continue

    res = run_openai_task(text=row['text'], entities=row['locations'])

    locations.append(res['function_response'])
    responses.append(res['model_response'])

  0%|          | 0/477 [00:00<?, ?it/s]

Expecting value: line 29 column 17 (char 477)
Expecting value: line 9 column 17 (char 170)


KeyboardInterrupt: 

In [239]:
len(locations)

433

In [241]:
locations[432]

['Асканія-Нова', 'Херсонській області', 'парку "Асканії-Нової"']

In [250]:
locations += raw_preds.loc[433:, 'locations'].to_list()

In [251]:
len(locations)

477

In [252]:
raw_preds['locations_new'] = locations

In [254]:
raw_preds.to_csv("../submissions/gpt_corrections_091.csv", index=None)

In [267]:
from json.decoder import JSONDecodeError

In [280]:
responses_fixed = []
for resp in responses:
    if 'error' in resp and type(resp['error']) == JSONDecodeError:
        responses_fixed.append({'error': str(resp['error'])})
    else:
        responses_fixed.append(resp)

In [281]:
json.dump(responses_fixed, fp=open('reponses.json', 'w'))

In [282]:
raw_preds

,text_id,locations,text,locations_new
0,0,[],"❗️Кількість поранених зросла до трьох, – Кличк...",[]
1,1,"[Києві, Шулявського шляхопроводу, Шулявського]","🥤В Києві за 91,13 млн гривень починаються робо...","[Києві, Шулявського шляхопроводу]"
2,2,"[Гоголеве, Миргородського району, Полтавської ...",▪️Сьогодні вночі росіяни завдали ракетного уда...,"[Гоголеве, Миргородського району, Полтавської ..."
3,3,[],Наразі у запасах росіян найбільше балістичних ...,[]
4,4,"[проспекті Академіка Глушкова, 9]","⛸В один день, 29 серпня, ДП ""Центральна учбово...",[]
...,...,...,...,...
472,472,"[Київщини, Україні, Київської області, Бучансь...",Допомога на відновлення: п’ять громад Київщини...,"[Київщини, Україні, Київської області, Бучансь..."
473,473,[Землі],Пророцтво Стругацьких: незвичайна бактерія Бра...,[Землі]
474,474,[],Із 1 вересня УГКЦ і ПЦУ перейшли на новий церк...,[]
475,475,"[Києві, Дніпра, Дніпро]",🦇 У Києві з Дніпра виловили іноземця в масці Б...,"[Києві, Дніпра, Дніпро]"


In [283]:
best_preds = pd.read_csv("../submissions/multibert_ukru_lingua_0906_ot_stoprows4_n.csv")

In [285]:
best_preds.loc[:433, 'locations'] = raw_preds.loc[:433, "locations_new"]

In [288]:
best_preds.to_csv("../submissions/multibert_ukru_lingua_0906_ot_stoprows4_n_gpt091.csv", index=None)